In [1]:
import boto3
import csv
import json
from jsonlines import Reader
from pprint import pprint

config = json.load(open('/home/tony/.aws/mycredentials'))
client = boto3.client('mturk', 
                      endpoint_url='https://mturk-requester-sandbox.us-east-1.amazonaws.com',
                      region_name=config['region_name'],
                      aws_access_key_id=config['aws_access_key_id'],
                      aws_secret_access_key=config['aws_secret_access_key'])

In [2]:
def get_hits(client):
    hits = client.list_hits(MaxResults=100)['HITs']
    
    # Load in hits into dictionary
    hit_dict = {}

    for hit in hits:
        hit_id = hit['HITId']
        hit_type_id = hit['HITTypeId']
        hit_layout_id = hit['HITLayoutId']
        hit_title = hit['Title']
        hit_group_id = hit['HITGroupId']
        context = hit['Question'].split('<p style="font-size: 12pt;font-family:Verdana;color: black;">')[1][1:15]
        if hit_title not in hit_dict:
            hit_dict[hit_title]  = {'hit_type_id': hit_type_id, 'hit_layout_id': hit_layout_id, 'hits': []}
        else:
            assert hit_dict[hit_title]['hit_type_id'] == hit_type_id
            assert hit_dict[hit_title]['hit_layout_id'] == hit_layout_id

        hit_dict[hit_title]['hits'].append((hit_id, context))

    return hit_dict

In [3]:
# hit_dict = get_hits(client)
qualifications = client.list_qualification_types(MustBeRequestable=True, MustBeOwnedByCaller=True)['QualificationTypes']
[e['QualificationTypeId'] for e in qualifications]

['3OY1URRDRXYSOP26ZM4ULWOVQ30TIE']

In [4]:
response = client.list_qualification_types(MustBeRequestable=True, MustBeOwnedByCaller=True)
assert response['ResponseMetadata']['HTTPStatusCode'] == 200
qual_dict = {d['Name']:d['QualificationTypeId'] for d in response['QualificationTypes']}

In [5]:
qual_dict

{'narrativeqa': '3OY1URRDRXYSOP26ZM4ULWOVQ30TIE'}

In [8]:
seen = set()

for line in Reader(open('../worker_scores.jsonl')):
    seen.add((line['worker_id'], line['qual_id']))

In [9]:
seen

{('A3F7KR3PHILUED', '3OY1URRDRXYSOP26ZM4ULWOVQ30TIE')}

In [ ]:
# # Create a HIT via API
# hit_type_id = hit_dict['Evaluating Question Answering (NarrativeQA)']['hit_type_id']
# hit_layout_id = hit_dict['Evaluating Question Answering (NarrativeQA)']['hit_layout_id']

# hit_layout_parameters = []

# with open('test_mt/narrativeqa_batch3.csv') as csvfile:
#     for line in csv.DictReader(csvfile):
#         for key, value in line.items():
#             hit_layout_parameters.append({'Name': key, 'Value': value})
            
#         client.create_hit_with_hit_type(HITTypeId=hit_type_id, 
#                                         HITLayoutId=hit_layout_id,
#                                         HITLayoutParameters=hit_layout_parameters,
#                                         LifetimeInSeconds=432000)
        
#         break

In [ ]:
# # Getting qualification request and accepting them
# response = client.list_qualification_requests()
# if response['ResponseMetadata']['HTTPStatusCode'] == 200:
#     for request in response['QualificationRequests']:
#         qual_request_id = request['QualificationRequestId']
#         worker_id = request['WorkerId']
#         qual_type_id = request['QualificationTypeId']
#         qual_name = client.get_qualification_type(QualificationTypeId=qual_type_id)['QualificationType']['Name']
#         print('Approving', 'A3F7KR3PHILUED',  'for', qual_name)
        
#         client.accept_qualification_request(QualificationRequestId=qual_request_id)

# # Remove qual from worker
# client.disassociate_qualification_from_worker(WorkerId='A3F7KR3PHILUED', QualificationTypeId='3ACE7X5EF9EID7V067SRRIKB9S7NEY')

# client.associate_qualification_with_worker(QualificationTypeId='3ACE7X5EF9EID7V067SRRIKB9S7NEY',
#                                            WorkerId='A3F7KR3PHILUED')

# client.list_workers_with_qualification_type(QualificationTypeId='3ACE7X5EF9EID7V067SRRIKB9S7NEY')

# # Remove qual from worker
# client.disassociate_qualification_from_worker(WorkerId='A3F7KR3PHILUED', QualificationTypeId='3ACE7X5EF9EID7V067SRRIKB9S7NEY')